In [1]:
import pandas as pd
df=pd.read_csv('quora_duplicate_questions.tsv',sep='\t')
df.dropna(inplace=True)
df=df.query('is_duplicate==1')
df.info()
df=df.iloc[50000:100000,:]

<class 'pandas.core.frame.DataFrame'>
Index: 149263 entries, 5 to 404286
Data columns (total 6 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   id            149263 non-null  int64 
 1   qid1          149263 non-null  int64 
 2   qid2          149263 non-null  int64 
 3   question1     149263 non-null  object
 4   question2     149263 non-null  object
 5   is_duplicate  149263 non-null  int64 
dtypes: int64(4), object(2)
memory usage: 8.0+ MB


In [2]:
df.drop(columns=['id','qid1','qid2',],inplace=True)

In [3]:
from transformers import pipeline
import tensorflow as tf
model = pipeline('text-classification', model='SamLowe/roberta-base-go_emotions', top_k=None)
positive_grad=['amusement','excitement','pride','optimism','gratitude','joy','admiration','love','approval','caring','desire','relief']
neutral_grad=['confusion','curiosity','realization','surprise','neutral']
negative_grad=['anger','disgust','grief','fear','sadness','nervousness','annoyance','disappointment','embarrassment','remorse','disapproval']
total_grad=positive_grad+neutral_grad+negative_grad

/opt/homebrew/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
def predict_emotion(text):
    model_output = model([text])
    predictions = model_output[0]
    
    max_score_index = max(range(len(predictions)), key=lambda i: predictions[i]["score"])
    return predictions[max_score_index]["label"]

In [5]:
from tqdm import tqdm

for index, row in tqdm(df.iterrows(), total=len(df), desc='Processing rows'):
    input_text = row['question1']
    target_text = row['question2']

    input_emotion = predict_emotion(input_text)
    target_emotion = predict_emotion(target_text)

    if (input_emotion == target_emotion or input_emotion == 'neutral' or target_emotion == 'neutral'):
        df.drop(index, inplace=True)
    else:
        if total_grad.index(input_emotion) > total_grad.index(target_emotion):
            df.at[index, 'question1'], df.at[index, 'question2'] = df.at[index, 'question2'], df.at[index, 'question1']
            input_emotion, target_emotion = target_emotion, input_emotion
        emotion_transition = f"{input_emotion} to {target_emotion}: "
        modified_input_text = emotion_transition + input_text
        df.at[index, 'question1'] = modified_input_text

Processing rows: 100%|██████████| 50000/50000 [1:02:16<00:00, 13.38it/s]


In [8]:
# df.drop(columns=['is_duplicate'],inplace=True)
df.to_csv('quora_new_2.csv',index=False)

In [ ]:
df.info()